In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras

/home/sfefilatyev/projects/tensorflow_exercises/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sfefilatyev/projects/tensorflow_exercises/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sfefilatyev/projects/tensorflow_exercises/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int1

In [2]:
housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [3]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=X_train.shape[1:]),
    keras.layers.Dense(1)
])
model.compile(loss="mean_squared_error", optimizer="sgd")
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))
mse_test = model.evaluate(X_test, y_test)
X_new = X_test[:3] # pretend these are new instances
y_pred = model.predict(X_new)
print(y_pred)

Train on 11610 samples, validate on 3870 samples
Epoch 1/20
11610/11610 [==============================] - 1s 94us/sample - loss: 0.8647 - val_loss: 0.6349
Epoch 2/20
11610/11610 [==============================] - 0s 26us/sample - loss: 0.7946 - val_loss: 1.4549
Epoch 3/20
11610/11610 [==============================] - 0s 26us/sample - loss: 0.7536 - val_loss: 0.5054
Epoch 4/20
11610/11610 [==============================] - 0s 27us/sample - loss: 0.4650 - val_loss: 0.4839
Epoch 5/20
11610/11610 [==============================] - 0s 25us/sample - loss: 0.4532 - val_loss: 0.4758
Epoch 6/20
11610/11610 [==============================] - 0s 26us/sample - loss: 0.4400 - val_loss: 0.4595
Epoch 7/20
11610/11610 [==============================] - 0s 26us/sample - loss: 0.4298 - val_loss: 0.4542
Epoch 8/20
11610/11610 [==============================] - 0s 26us/sample - loss: 0.4275 - val_loss: 0.4445
Epoch 9/20
11610/11610 [==============================] - 0s 26us/sample - loss: 0.4206 - val_l

In [5]:
# Wide and deep neural net.
input_ = keras.layers.Input(shape=X_train.shape[1:])
hidden1 = keras.layers.Dense(30, activation="relu")(input_)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.Concatenate()([input_, hidden2])
output = keras.layers.Dense(1)(concat)
model2 = keras.Model(inputs=[input_], outputs=[output])
model2.compile(loss="mean_squared_error", optimizer="Adam")
print(model2.summary())
history2 = model2.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))
mse_test2 = model2.evaluate(X_test, y_test)
X_new = X_test[:3] # pretend these are new instances
y_pred = model2.predict(X_new)
print(y_pred)

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 8)]          0                                            
__________________________________________________________________________________________________
dense_5 (Dense)                 (None, 30)           270         input_2[0][0]                    
__________________________________________________________________________________________________
dense_6 (Dense)                 (None, 30)           930         dense_5[0][0]                    
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 38)           0           input_2[0][0]                    
                                                                 dense_6[0][0]              

In [14]:
input_A = keras.layers.Input(shape=[5], name="wide_input")
input_B = keras.layers.Input(shape=[6], name="deep_input")
hidden1 = keras.layers.Dense(30, activation="relu")(input_B)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.concatenate([input_A, hidden2])
output = keras.layers.Dense(1, name="output")(concat)
model3 = keras.Model(inputs=[input_A, input_B], outputs=[output])
model3.compile(loss="mse", optimizer=keras.optimizers.SGD(lr=1e-3))

X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]
X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]
history = model3.fit((X_train_A, X_train_B), y_train, epochs=20,
                    validation_data=((X_valid_A, X_valid_B), y_valid))
mse_test = model3.evaluate((X_test_A, X_test_B), y_test)
y_pred = model3.predict((X_new_A, X_new_B))

Train on 11610 samples, validate on 3870 samples
Epoch 1/20
11610/11610 [==============================] - 0s 38us/sample - loss: 1.9651 - val_loss: 0.8567
Epoch 2/20
11610/11610 [==============================] - 0s 29us/sample - loss: 0.7392 - val_loss: 0.7320
Epoch 3/20
11610/11610 [==============================] - 0s 29us/sample - loss: 0.6713 - val_loss: 0.6932
Epoch 4/20
11610/11610 [==============================] - 0s 30us/sample - loss: 0.6390 - val_loss: 0.6658
Epoch 5/20
11610/11610 [==============================] - 0s 30us/sample - loss: 0.6131 - val_loss: 0.6418
Epoch 6/20
11610/11610 [==============================] - 0s 29us/sample - loss: 0.5921 - val_loss: 0.6238
Epoch 7/20
11610/11610 [==============================] - 0s 31us/sample - loss: 0.5727 - val_loss: 0.6043
Epoch 8/20
11610/11610 [==============================] - 0s 29us/sample - loss: 0.5586 - val_loss: 0.5912
Epoch 9/20
11610/11610 [==============================] - 0s 30us/sample - loss: 0.5443 - val_l

In [18]:
model3.layers

In [19]:
model3.layers[1]

In [20]:
hidden1 = model3.layers[1]

In [21]:
hidden1.name

'dense_12'

In [23]:
model3.get_layer('dense_12') is hidden1

True

In [28]:
main_output = keras.layers.Dense(1, name="main_output")(concat)
aux_output = keras.layers.Dense(1, name="aux_output")(hidden2)
model4 = keras.Model(inputs=[input_A, input_B], outputs=[main_output, aux_output])
model4.compile(loss=["mse", "mse"], loss_weights=[0.9, 0.1], optimizer=keras.optimizers.SGD(lr=1e-3))

history = model4.fit((X_train_A, X_train_B), [y_train, y_train], epochs=20,
                    validation_data=((X_valid_A, X_valid_B), [y_valid, y_valid]))
total_loss, main_loss, aux_loss = model4.evaluate([X_test_A, X_test_B], [y_test, y_test])
y_pred_main, y_pred_aux = model4.predict([X_new_A, X_new_B])
print(total_loss, main_loss, aux_loss)
print(y_pred_main, y_pred_aux)

Train on 11610 samples, validate on 3870 samples
Epoch 1/20
11610/11610 [==============================] - 1s 48us/sample - loss: 1.1455 - main_output_loss: 1.0176 - aux_output_loss: 2.2944 - val_loss: 0.6432 - val_main_output_loss: 0.5613 - val_aux_output_loss: 1.3784
Epoch 2/20
11610/11610 [==============================] - 0s 36us/sample - loss: 0.5632 - main_output_loss: 0.5097 - aux_output_loss: 1.0443 - val_loss: 0.5633 - val_main_output_loss: 0.5187 - val_aux_output_loss: 0.9642
Epoch 3/20
11610/11610 [==============================] - 0s 34us/sample - loss: 0.5232 - main_output_loss: 0.4840 - aux_output_loss: 0.8760 - val_loss: 0.5434 - val_main_output_loss: 0.5044 - val_aux_output_loss: 0.8937
Epoch 4/20
11610/11610 [==============================] - 0s 33us/sample - loss: 0.5096 - main_output_loss: 0.4728 - aux_output_loss: 0.8397 - val_loss: 0.5349 - val_main_output_loss: 0.4973 - val_aux_output_loss: 0.8720
Epoch 5/20
11610/11610 [==============================] - 0s 33us/s